In [1]:
#Libraries
import numpy as np
import pandas as pd
import numpy_financial as npf
from datetime import date

In [2]:
#30-Year FIXED RATE
def fixed_rate_mortgage(interest: float, years: int, payments_year: int, mortgage: int, start_date: str):
  periods = years * payments_year
  df = pd.DataFrame(index=range(1, periods+1))
  df["Date"] = pd.date_range(start_date, periods=periods, freq='MS', name='Payment Date').date
  df["Payment"] = -1 * npf.pmt(interest/12, periods,mortgage)
  df["Interest Paid"] = -1 * npf.ipmt(interest/payments_year, df.index, periods,mortgage)
  df["Principal Paid"] = -1 * npf.ppmt(interest/payments_year, df.index, periods,mortgage)
  df['Ending Balance'] = mortgage - df['Principal Paid'].cumsum()
  df[df['Ending Balance'] < 0] = 0
  total_loan = loan + df['Interest Paid'].sum()
  with pd.option_context(
      'display.max_columns', None,
      'display.float_format', '${:,.2f}'.format):
    print(df)
  print(f"\nTotal of the loan is: ${total_loan:,.2f}.\n")
  return df

Here we enter the inputs for mortgage plan and run the function for generation of payment plan:

In [3]:
interest = 0.04
years = 30
payments_year = 12

#loan = input("Write the amount you want to loan \n")
#loan = float(loan)
loan = 1000000

#start_date = input("What is the start date, in YYYY-MM-DD format")
start_date = "2001-01-01"
year, month, day = map(int, start_date.split("-"))
start_date = date(year, month, day)

mydf = fixed_rate_mortgage(interest, years, payments_year, loan, start_date)

           Date   Payment  Interest Paid  Principal Paid  Ending Balance
1    2001-01-01 $4,774.15      $3,333.33       $1,440.82     $998,559.18
2    2001-02-01 $4,774.15      $3,328.53       $1,445.62     $997,113.56
3    2001-03-01 $4,774.15      $3,323.71       $1,450.44     $995,663.12
4    2001-04-01 $4,774.15      $3,318.88       $1,455.28     $994,207.84
5    2001-05-01 $4,774.15      $3,314.03       $1,460.13     $992,747.71
..          ...       ...            ...             ...             ...
356  2030-08-01 $4,774.15         $78.78       $4,695.37      $18,938.53
357  2030-09-01 $4,774.15         $63.13       $4,711.02      $14,227.50
358  2030-10-01 $4,774.15         $47.43       $4,726.73       $9,500.78
359  2030-11-01 $4,774.15         $31.67       $4,742.48       $4,758.29
360  2030-12-01 $4,774.15         $15.86       $4,758.29           $0.00

[360 rows x 5 columns]

Total of the loan is: $1,718,695.06.



In [4]:
mydf.to_csv(f"Montly_Payment_Plan_IR={interest}_YR={years}_30yr.csv")
mydf

,Date,Payment,Interest Paid,Principal Paid,Ending Balance
1,2001-01-01,4774.152955,3333.333333,1440.819621,9.985592e+05
2,2001-02-01,4774.152955,3328.530601,1445.622353,9.971136e+05
3,2001-03-01,4774.152955,3323.711860,1450.441095,9.956631e+05
4,2001-04-01,4774.152955,3318.877056,1455.275898,9.942078e+05
5,2001-05-01,4774.152955,3314.026137,1460.126818,9.927477e+05
...,...,...,...,...,...
356,2030-08-01,4774.152955,78.779671,4695.373283,1.893853e+04
357,2030-09-01,4774.152955,63.128427,4711.024527,1.422750e+04
358,2030-10-01,4774.152955,47.425012,4726.727943,9.500776e+03
359,2030-11-01,4774.152955,31.669252,4742.483702,4.758292e+03


In [5]:
#20-Year FIXED RATE

In [6]:
interest = 0.025
years = 20
payments_year = 12
mortgage = 1000000
start_date = (date(2021, 1, 1))

In [7]:
rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
rng.name = "Payment Date"
df = pd.DataFrame(index=rng, columns=['Payment', 'Principal Paid', 'Interest Paid', 'Ending Balance'], dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"

In [8]:
df["Payment"] = -1 * npf.pmt(interest/12, years*payments_year,mortgage)
df["Interest Paid"] = -1 * npf.ipmt(interest/payments_year, df.index, years*payments_year,mortgage)
df["Principal Paid"] = -1 * npf.ppmt(interest/payments_year, df.index, years*payments_year,mortgage)
df["Ending Balance"] = 0
df.loc[1, "Ending Balance"] = mortgage - df.loc[1, "Principal Paid"]
df = df.round(2)

In [9]:
for period in range(2, len(df)+1):
    previous_balance = df.loc[period-1, "Ending Balance"]
    principal_paid = df.loc[period, "Principal Paid"]
    
    if previous_balance == 0:
        df.loc[period, ['Payment', 'Principal Paid', 'Interest Paid', 'Ending Balance']] == 0
        continue
    elif principal_paid <= previous_balance:
        df.loc[period, "Ending Balance"] = previous_balance - principal_paid
        
print(df)

       Payment Date  Payment  Principal Paid  Interest Paid  Ending Balance
Period                                                                     
1        2021-01-01  5299.03         3215.70        2083.33       996784.30
2        2021-02-01  5299.03         3222.39        2076.63       993561.91
3        2021-03-01  5299.03         3229.11        2069.92       990332.80
4        2021-04-01  5299.03         3235.84        2063.19       987096.96
5        2021-05-01  5299.03         3242.58        2056.45       983854.38
...             ...      ...             ...            ...             ...
236      2040-08-01  5299.03         5244.17          54.85        21086.21
237      2040-09-01  5299.03         5255.10          43.93        15831.11
238      2040-10-01  5299.03         5266.05          32.98        10565.06
239      2040-11-01  5299.03         5277.02          22.01         5288.04
240      2040-12-01  5299.03         5288.01          11.02            0.03

[240 rows x

In [10]:
df.to_csv(f"Montly_Payment_Plan_IR={interest}_YR={years}_20yr.csv")

In [11]:
def fixed_rate_mortgage2(interest: float, years: int, payments_year: int, mortgage: int, start_date: str):
    periods = years * payments_year
    df = pd.DataFrame(index=range(1, periods+1))
    df["Date"] = pd.date_range(start_date, periods=periods, freq='MS', name='Payment Date').date
    df["Interest Rate"] = interest
    # -IPMT([@[ARM 7/1]]/12;1;DurationOfLoan-ROWS($C$4:C4)+1;
    # -IPMT([@[ARM 7/1]]/12;1;DurationOfLoan-ROWS($C$4:C6)+1;[@[UPB start]])
    df["Interest Paid"] = -1 * npf.ipmt(df["Interest Rate"]/payments_year, 132, periods, mortgage)
    #df["Interest Paid"] = -1 * npf.ipmt(df["Interest Rate"]/payments_year, df.index, years*payments_year-df.index+1, mortgage)
    # -PPMT([@[ARM 7/1]]/12;1;DurationOfLoan-ROWS($C$4:C13)+1
    df["Principal Paid"] = -1 * npf.ppmt(df["Interest Rate"]/payments_year, 132, periods, mortgage)
    #df["Principal Paid"] = -1 * npf.ppmt(df["Interest Rate"]/payments_year, df.index, years*payments_year-df.index+1, mortgage)
    #df["Payment"]        = -1 * npf.pmt( df["Interest Rate"]/payments_year, periods, mortgage)
    df["Payment"]        = df["Principal Paid"] + df["Interest Paid"]
    
    df['Ending Balance'] = mortgage - df['Principal Paid'].cumsum()
    df[df['Ending Balance'] < 0] = 0
    total_loan = loan + df['Interest Paid'].sum()
    print(df)
    print(f"\nTotal of the loan is: ${total_loan:,.2f}.\n")
    return df

In [12]:
#For 20 years yearly averaged rates, the following are assumed.
movingRates = [0.08033, # year 2001
               0.08033, # year 2002
               0.08033, # year 2003
               0.08033, # year 2004
               0.08033, # year 2005
               0.08033, # year 2006
               0.08033, # year 2007
               0.06337, # year 2008
               0.06027, # year 2009
               0.05037, # year 2010
               0.04690, # year 2011
               0.04448, # year 2012
               0.03658, # year 2013
               0.03976, # year 2014
               0.04162, # year 2015
               0.03851, # year 2016
               0.03654, # year 2017
               0.03990, # year 2018
               0.04545, # year 2019
               0.03936  # year 2020 
              ]
ARM71Rates = np.repeat(movingRates, 12)   # convert yearly rates to montly repetitions

interest = ARM71Rates
years = 20
payments_year = 12

#loan = input("Write the amount you want to loan \n")
#loan = float(loan)
loan = 1000000

#start_date = input("What is the start date, in YYYY-MM-DD format")
start_date = "2001-01-01"
year, month, day = map(int, start_date.split("-"))
start_date = date(year, month, day)

In [13]:
mydf = fixed_rate_mortgage2(interest, years, payments_year, loan, start_date)

           Date  Interest Rate  Interest Paid  Principal Paid      Payment  \
1    2001-01-01        0.08033    4332.969981     4051.980116  8384.950096   
2    2001-02-01        0.08033    4332.969981     4051.980116  8384.950096   
3    2001-03-01        0.08033    4332.969981     4051.980116  8384.950096   
4    2001-04-01        0.08033    4332.969981     4051.980116  8384.950096   
5    2001-05-01        0.08033    4332.969981     4051.980116  8384.950096   
..          ...            ...            ...             ...          ...   
236  2020-08-01        0.03936    1808.936467     4217.196347  6026.132814   
237  2020-09-01        0.03936    1808.936467     4217.196347  6026.132814   
238  2020-10-01        0.03936    1808.936467     4217.196347  6026.132814   
239  2020-11-01        0.03936    1808.936467     4217.196347  6026.132814   
240  2020-12-01        0.03936    1808.936467     4217.196347  6026.132814   

     Ending Balance  
1     995948.019884  
2     991896.039769

In [14]:
mydf.to_csv(f"Montly_Payment_Plan_IR=ARM71_YR={years}_20yrvar.csv")